In [ ]:
!wget https://raw.githubusercontent.com/openforcefield/2023-workshop-vignettes/master/colab_setup.ipynb
%run colab_setup.ipynb

# Lipid Self-assembly

In [1]:
from io import StringIO

from openff.toolkit import Topology, Molecule, ForceField
from openff.interchange import Interchange
from openff.interchange.components._packmol import pack_box
from openff.units import unit

import nglview
import openmm
import numpy as np
import mdtraj

def visualize(topology):
    """Visualize a topology with nglview"""
    with StringIO() as f:
        topology.to_file(file=f)
        pdb_str = f.getvalue()
    return nglview.show_text(pdb_str)


The OpenEye Toolkits are found to be installed but not licensed and therefore will not be used.
The OpenEye Toolkits require a (free for academics) license, see https://docs.eyesopen.com/toolkits/python/quickstart-python/license.html


In [35]:
#TODO: phosphatidylethanolamine or phosphatidylserine may have easier-to-spot hydrophillic head groups
dlpc = Molecule.from_smiles("CCCCCCCCCCCC(=O)OC[C@H](CO[P@](=O)([O-])OCC[N+](C)(C)C)OC(=O)CCCCCCCCCCC")
lipids = [dlpc]

conc_nacl = 0.1 * unit.mole/unit.liter
n_waters = 4000
n_lipids = [25]
target_density = 1.0 * unit.gram / unit.milliliter

In [36]:
water = Molecule.from_smiles("O")
na = Molecule.from_smiles("[Na+]")
cl = Molecule.from_smiles("[Cl-]")

molarity_pure_water = 55.5 * unit.mole / unit.liter
n_nacl = int((n_waters / molarity_pure_water * conc_nacl).to(unit.dimensionless).m)
molecules = [*lipids, water, na, cl]
n_copies = [*n_lipids, n_waters, n_nacl, n_nacl]

total_mass = sum([sum([atom.mass for atom in molecule.atoms]) * n for molecule, n in zip(molecules, n_copies)])
target_volume = total_mass / target_density
box_size = np.ones(3) * np.cbrt(target_volume)

traj, resname = pack_box(
    molecules,
    n_copies,
    box_size = box_size,
    tolerance=0.05 * unit.nanometer,
)

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 15, name: , idx: 19, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 18, aromatic: False, chiral: False
bond order: 2, chiral: False to atom atomic num: 8, name: , idx: 20, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 21, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 22, aromatic: False, chiral: False

Warning (not error because allow_undefined_stereo=True): RDMol has unspecified stereochemistry. Undefined chiral centers are:
 - Atom P (index 46)

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 15, name: , idx: 46, aromatic: False, chiral: True with bonds:
bond order: 1, 

In [37]:
top = Topology.from_mdtraj(traj.topology, unique_molecules=[dlpc, water, na, cl])
top.set_positions(traj.xyz.reshape(-1, 3) * unit.nanometer)
top.box_vectors = traj.unitcell_vectors.reshape(3, 3) * unit.nanometer

In [38]:
w = visualize(top)
w.add_representation("line", selection="water")
w

NGLWidget()

In [39]:
with open("topology.json", "w") as f:
    print(top.to_json(), file=f)
top.to_file("topology.pdb", file_format="PDB")

In [40]:
sage = ForceField("openff-2.1.0.offxml")

In [41]:
interchange = Interchange.from_smirnoff(sage, top)

In [42]:
with open("interchange.json", "w") as f:
    f.write(interchange.json())

In [43]:
openmm_system = interchange.to_openmm()
openmm_topology = interchange.to_openmm_topology()
openmm_positions = interchange.positions.to_openmm()

temperature = 300 * openmm.unit.kelvin

openmm_system.addForce(
    openmm.MonteCarloBarostat(
        1.0 * openmm.unit.bar, 
        temperature,
    )
)

4

In [44]:
delta_t = 2 * unit.femtoseconds

# Construct and configure a Langevin integrator at 300 K with an appropriate friction constant and time-step
integrator = openmm.LangevinMiddleIntegrator(
    temperature,
    1 / openmm.unit.picosecond,
    delta_t.to_openmm(),
)

# Combine the topology, system, integrator and initial positions into a simulation
simulation = openmm.app.Simulation(openmm_topology, openmm_system, integrator)
simulation.context.setPositions(openmm_positions)

# Add a reporter to record the structure every data_freq steps
data_freq = 1000
dcd_reporter = openmm.app.DCDReporter("trajectory.dcd", data_freq)
simulation.reporters.append(dcd_reporter)

state_data_reporter = openmm.app.StateDataReporter(
    "data.csv",
    data_freq,
    step=True,
    potentialEnergy=True,
    temperature=True,
    density=True,
)
simulation.reporters.append(state_data_reporter)

In [45]:
simulation.minimizeEnergy(
    tolerance=openmm.unit.Quantity(
        value=50.0, unit=openmm.unit.kilojoule_per_mole / openmm.unit.nanometer
    )
)
minimized_state = simulation.context.getState(
    getPositions=True, getEnergy=True, getForces=True
)

print(
    "Minimised to",
    minimized_state.getPotentialEnergy(),
    "with maximum force",
    max(
        np.sqrt(v.x * v.x + v.y * v.y + v.z * v.z) for v in minimized_state.getForces()
    ),
    minimized_state.getForces().unit.get_symbol(),
)

minimized_coords = minimized_state.getPositions()

Minimised to -191555.3813474467 kJ/mol with maximum force 2637.890863463706 kJ/(nm mol)


In [46]:
simulation.context.setVelocitiesToTemperature(temperature)

timing_walltime = 10.0 * unit.second

simulation.runForClockTime(timing_walltime.to_openmm())

slowdown_factor = (
    simulation.currentStep 
    * delta_t 
    / timing_walltime
).to(unit.nanosecond/unit.hour)

print(f"{simulation.currentStep} steps in {timing_walltime} ({slowdown_factor})")

5260 steps in 10.0 second (3.7872 nanosecond / hour)


In [55]:
simulation_time = 10 * unit.nanosecond
steps = round((simulation_time / delta_t).to(unit.dimensionless).m)
simulation.step(steps)

## Visualisation

In [8]:
import numpy as np
import mdtraj
import nglview

def wrap(trajectory):
    """Wrap positions back into the central box."""
    positions = trajectory.xyz[..., None, :]
    box = trajectory.unitcell_vectors[:, None, :, :]
    
    frac_coords = positions @ np.linalg.inv(box)
    wrapped_positions = (frac_coords - np.floor(frac_coords)) @ box
    assert wrapped_positions.shape[2] == 1
        
    return mdtraj.Trajectory(
        wrapped_positions[:, :, 0, :],
        trajectory.top,
        trajectory.time,
        trajectory.unitcell_lengths,
        trajectory.unitcell_angles,
    )
mdtraj.Trajectory.wrap = wrap

# mdtraj_top = mdtraj.Topology.from_openmm(top.to_openmm())
mdtraj_top = mdtraj.load("topology.pdb").top

trajectory: mdtraj.Trajectory = mdtraj.load(
    "trajectory.dcd", top=mdtraj_top, stride=10
)


In [9]:
lipid_resnames = set()
lipid_idcs = []
for res in mdtraj_top.residues:
    if res.name.upper() not in ['HOH', 'CL-', 'NA+']:
        lipid_resnames.add(res.name)
        lipid_idcs.extend(atom.index for atom in res.atoms)
        

In [10]:
final_frame_lipid_centroid = trajectory.xyz[-1][lipid_idcs].sum(axis=0) / len(lipid_idcs)
box_centers = trajectory.unitcell_vectors.sum(axis=-1)/2

In [11]:
trajectory.xyz = trajectory.xyz - final_frame_lipid_centroid + box_centers[:, None, :]
trajectory = trajectory.wrap().make_molecules_whole()

In [32]:
view = nglview.show_mdtraj(trajectory)
lipid_resnames_selection = ' OR '.join(lipid_resnames)
view.clear()
view.add_representation("spacefill", selection=f"not ({lipid_resnames_selection})", opacity=0.2)
view.add_representation("spacefill", selection=lipid_resnames_selection)
# view.add_unitcell()
# view.add_axes()
view

NGLWidget(max_frame=500)

In [49]:
n_frames = view.max_frame + 1
fps = 60
seconds_per_frame = 1 / fps

view.frame = 0
time.sleep(10)
for i in range(1, n_frames, 1):
    view.frame = i
    time.sleep(seconds_per_frame)
    # view.control.spin([1,1,0], 0.1)
    